In [2]:
import meshzoo
import numpy as np
import igl
import meshplot as mp
import scipy.io
import os
import math
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import Image, display, Markdown
import ipywidgets as widgets
from meshplot_utils import add_transparent_mesh, color4plot, Out
#from utils.debug_utils import picture_table_with_title
import utils.debug_utils  as du
import utils.geometry  as gu
import meshplot_utils as mu 
import itertools

#reload packes without restarting notebook 
import importlib
importlib.reload(gu)

#help functions
def reflectX(xyz):
    if len(xyz.shape)==2:
        return  np.concatenate( (-xyz[:,0:1], xyz[:,1:3]), axis=1) 
    else:
        return  np.concatenate( (-xyz[0:1], xyz[1:3]) ) 

def get_bboxes(vert):
    m = np.min(vert, axis=0)
    ma = np.max(vert, axis=0)
    ma_half = np.max(vert, axis=0)
    ma_half[0] = (m[0]+ma_half[0])/2.0

    # Corners of the bounding box
    v_box = np.array([[m[0], m[1], m[2]], [ma[0], m[1], m[2]], [ma[0], ma[1], m[2]], [m[0], ma[1], m[2]],
                    [m[0], m[1], ma[2]], [ma[0], m[1], ma[2]], [ma[0], ma[1], ma[2]], [m[0], ma[1], ma[2]]])


    v_box_half = np.array([[m[0], m[1], m[2]], [ma_half[0], m[1], m[2]], [ma_half[0], ma_half[1], m[2]], [m[0], ma_half[1], m[2]],
                    [m[0], m[1], ma_half[2]], [ma_half[0], m[1], ma_half[2]], [ma_half[0], ma_half[1], ma_half[2]], [m[0], ma_half[1], ma_half[2]]])                  

    # Edges of the bounding box
    f_box = np.array([[0, 1], [1, 2], [2, 3], [3, 0], [4, 5], [5, 6], [6, 7], 
                      [7, 4], [0, 4], [1, 5], [2, 6], [7, 3]], dtype=np.int)
                      
    return v_box, v_box_half, f_box

def draw_bboxes(p, vert):
    v_box, v_box_half, f_box = get_bboxes(vert)
    
    id1 = p.add_edges(v_box, f_box, shading={"line_color": "black"})
    id2 = p.add_edges(v_box_half, f_box, shading={"line_color": "black",'line_width':0.3})
    id3 = p.add_edges(v_box, np.array([[0, 1]],dtype=np.int), shading={"line_color": "red",'line_width':1})
    id4 = p.add_edges(v_box, np.array([[0, 3]],dtype=np.int), shading={"line_color": "green",'line_width':1})
    id5 = p.add_edges(v_box, np.array([[0, 4]],dtype=np.int), shading={"line_color": "blue",'line_width':1})
    
    return [id1,id2,id3,id4,id5]

sphere_plot_ids = []
def visualize_sphere(p, V, selected_indices, selected_col):
    res = 0
    #print(f'(V,F): {(V.shape,F.shape)}')
    for i in sphere_data.keys():
        #print((sphere_data[i]['V'].shape, sphere_data[i]['F'].shape))
        if sphere_data[i]['V'].shape[0] ==V.shape[0] and sphere_data[i]['F'].shape[0] == F.shape[0] and  not np.any(sphere_data[i]['F'] - F):
            print(f"resolution level={i}")
            res = i
            break

    S = sphere_data[res]
    sphere_size = S['V'][:,0].max() - S['V'][:,0].min()
    shape_scale = V[:,0].max() - V[:,0].min()
    SV = (S['V']/sphere_size)*shape_scale + np.array( [V[:,0].max() + shape_scale, 0, 0]) #normlaize to mesh size and shift left 
    if reflect_sphere_y:
        SV[:,1] = -SV[:,1]
    sphere_plot_ids.append(p.add_mesh(SV,S['F'], shading = shading_dic))
    #selected_vert = np.stack( (SV[vr_max,:],SV[vl_max,:]), axis =1 ).transpose()
    selected_vert = SV[selected_indices,:]
    sphere_plot_ids.append( p.add_points(selected_vert, c =selected_col, shading={"point_size": 0.15}) )
    sphere_plot_ids.extend(draw_bboxes(p, SV))
    return sphere_plot_ids

def Out(str):
    display(Markdown(f'### {str}'))


# symmetry configurations 
# shading_dic = dict(wireframe=True, line_color= [0.2,0.2,0.2] )
# resolutions= [2,3,4]

npy_file     = '../auxilary/icoshpere_meshes.npy'
npy_sub_file = '../auxilary/icoshpere_mesh_subdivision_1.npy'
 

#visualize_sphere = True 
sphere_data = []
#if visualize_sphere:
sphere_data = np.load(npy_sub_file, allow_pickle = True).item()
reflect_sphere_y = True #MCMR intializes shapes with eco-sphere reflected along y axis 
meshplot_to_html = True
show_sphere_bydefault = True
show_image_table = True #show input and rendered  rgb and mask images 

enable_index_symmetry     = True  #symmetry of sphere deformation 

sphere_plot_ids = []
def get_sphere(V):
    res = 0
    #print(f'(V,F): {(V.shape,F.shape)}')
    for i in sphere_data.keys():
        #print((sphere_data[i]['V'].shape, sphere_data[i]['F'].shape))
        if sphere_data[i]['V'].shape[0] ==V.shape[0] and sphere_data[i]['F'].shape[0] == F.shape[0] and  not np.any(sphere_data[i]['F'] - F):
            print(f"resolution level={i}")
            res = i
            break

    S = sphere_data[res]
    sphere_size = S['V'][:,0].max() - S['V'][:,0].min()
    shape_scale = V[:,0].max() - V[:,0].min()
    #SV = (S['V']/sphere_size)*shape_scale + np.array( [V[:,0].max() + shape_scale, 0, 0]) #normlaize to mesh size and shift left 
    SV = (S['V']/sphere_size)*shape_scale #normlaize to mesh size and shift left 
    if reflect_sphere_y:
        SV[:,1] = -SV[:,1]
    #sphere_plot_ids.append(p.add_mesh(SV,S['F'], shading = shading_dic))
    return SV, S['F']

In [3]:
#configurations 
shading_dic = dict(wireframe=True, line_color= [0.2,0.2,0.2] )
resolutions= [2,3,4]

npy_file     = '../auxilary/icoshpere_meshes.npy'
npy_sub_file = '../auxilary/icoshpere_mesh_subdivision_1.npy'


#=============================full test datas ======================================
data_path_list = ['../save/car_10_MRCNN__/car-qualitative',  \
                 '../save/plane_car_1_PointRend__/aeroplane-car-qualitative', \
                 '../save/plane_car_2_PointRend__/aeroplane-car-qualitative', \
                 '../save/bicycle_bus_car_bike_1_PointRend__/bicycle-bus-car-motorbike-qualitative', \
                 '../save/bicycle_bus_car_bike_4_PointRend__/bicycle-bus-car-motorbike-qualitative' \
                ]


#widgets.Checkbox(value=False,description='show initial shape')
  

#visualize_sphere = True 
sphere_data = []
#if visualize_sphere:
sphere_data = np.load(npy_sub_file, allow_pickle = True).item()
reflect_sphere_y = True #MCMR intializes shapes with eco-sphere reflected along y axis 
meshplot_to_html = True
show_sphere_bydefault = True
show_image_table = True #show input and rendered  rgb and mask images 

enable_index_symmetry     = True  #symmetry of sphere deformation 
enable_Hausdorff_symmetry = True   #symmetry of reconstructed mesh  (enable only one to avoid  plots being messed up)

In [4]:
data_path_list = ['../save/plane_car_1_PointRend__old/aeroplane-car-qualitative', \
                  '../save/plane_car_2_PointRend__old/aeroplane-car-qualitative'  \
                ]
data_path = '../save/plane_car_2_PointRend__old/aeroplane-car-qualitative'
data = np.load( data_path + '/shape_symmetries.npy', allow_pickle = True).item()
print(data.keys())
print(data['LR_max'].shape[0])



data_path = '../save/plane_car_2_PointRend__old/aeroplane-car-qualitative'
data = np.load( data_path + '/shape_symmetries.npy', allow_pickle = True).item()
print(data.keys())
print(data['LR_max'].shape[0])

data_path = '../save/plane_car_2_PointRend__old/aeroplane-car-qualitative'
data = np.load( data_path + '/shape_symmetries.npy', allow_pickle = True).item()

#fetching saved data 
data = np.load( data_path + '/shape_symmetries.npy', allow_pickle = True).item()
LR_max = data['LR_max']
V_right = data['V_right']
V_opposite = data['V_opposite']
H_symm = data['H_symm']
i_max = LR_max.argmax()
ih_max = H_symm.argmax()
print(f'Reported errors :')
print(f'    Max-Max Symmetry Errors  =  {LR_max.max():.4f}  for sample {i_max:04}')
print(f'    Mean-Max Symmetry error = { data["LR_max"].sum()/data["LR_max"].shape[0]}')
print(f'    Mean-Mean Symmetry error = { data["LR_mean"].sum()/data["LR_mean"].shape[0]}\n')

print(f'    Mean Hausdorff  Symmetry error = { H_symm.sum()/H_symm.shape[0]}')
print(f'    Max Hausdorff  Symmetry error =   {H_symm.max():.4f} for sample {ih_max:04}\n')


# load mesh with max error
class_name = os.path.split(data_path)[-1].split("-")[0]
obj_file = f'{data_path}/obj/{class_name}_{i_max:04}.obj'
V0, F = igl.read_triangle_mesh(obj_file)
V = V0 -  np.mean(V0,axis=0) #align shape to 0,0,0  

#recompute relative symmetry errors 
LR_diff_unscaled = np.linalg.norm(V[V_right,:] - reflectX(V[V_opposite[V_right],:]), axis=1)
shape_scale = V[:,0].max() - V[:,0].min()
LR_diff = LR_diff_unscaled/shape_scale
symm_error = np.zeros(V.shape[0])
symm_error[V_right] = LR_diff
symm_error[V_opposite[V_right]] = symm_error[V_right] 

print(f'Recompute errors after aligment (deviation={100*np.mean(V0,axis=0)/shape_scale}%) :')
print(f'    Max Symmetry error  = {100*LR_diff.max():.3f}% of shape width (on vertex {LR_diff.argmax()} )')
print(f'    Max Symmetry error  = {100*symm_error.max():.3f}% of shape width (on vertex {symm_error.argmax()} )')
print(f'    Mesh with max error in the session:  {obj_file}\n')

Hobj_file = f'{data_path}/obj/{class_name}_{ih_max:04}.obj'
mu.Out(f'### plot of max Hausdorff symmetry for {Hobj_file}')

## mesh plot 
VH0, FH = igl.read_triangle_mesh(Hobj_file)
VH = VH0 -  np.mean(VH0,axis=0) #align shape to 0,0,0  
VH_ref = reflectX(VH)
sizeX = VH[:,0].max() - VH[:,0].min() 

enable_drawing = False
if enable_drawing:
  ph = mp.plot(VH ,FH,  shading = shading_dic)
  add_transparent_mesh(ph, VH + np.array([1.5*sizeX,0,0]) ,FH, c = color4plot([1,1,0],VH) , shading = shading_dic, opacity=0.5)

  ph = mp.plot(VH ,FH,  shading = shading_dic)
  add_transparent_mesh(ph, VH_ref + np.array([1.5*sizeX,0,0]), F, c = color4plot([0,0,1],VH) , shading = shading_dic, opacity=0.3)


  ph = mp.plot(VH ,FH,  shading = shading_dic)
  add_transparent_mesh(ph, VH + np.array([1.5*sizeX,0,0]) ,FH, c = color4plot([1,1,0],VH) , shading = shading_dic, opacity=0.5)
  add_transparent_mesh(ph, VH_ref + np.array([1.5*sizeX,0,0]), F, c = color4plot([0,0,1],VH) , shading = shading_dic, opacity=0.3)

dict_keys(['LR_mean', 'LR_max', 'H_symm', 'V_right', 'V_opposite'])
477
dict_keys(['LR_mean', 'LR_max', 'H_symm', 'V_right', 'V_opposite'])
477
Reported errors :
    Max-Max Symmetry Errors  =  0.2717  for sample 0097
    Mean-Max Symmetry error = 0.13859639622080505
    Mean-Mean Symmetry error = 0.02048480094653865

    Mean Hausdorff  Symmetry error = 0.026586508398024386
    Max Hausdorff  Symmetry error =   0.0942 for sample 0063

Recompute errors after aligment (deviation=[-0.07300931 -3.02792218 -3.86877649]%) :
    Max Symmetry error  = 27.303% of shape width (on vertex 62 )
    Max Symmetry error  = 27.303% of shape width (on vertex 124 )
    Mesh with max error in the session:  ../save/plane_car_2_PointRend__old/aeroplane-car-qualitative/obj/aeroplane_0097.obj



### ### plot of max Hausdorff symmetry for ../save/plane_car_2_PointRend__old/aeroplane-car-qualitative/obj/aeroplane_0063.obj

In [5]:
symmetry_data = data 
V_right    = symmetry_data['V_right']
V_opposite = symmetry_data['V_opposite']
V_left = V_opposite[V_right]
V_middle   = np.nonzero(np.arange(0,V_opposite.shape[0]) == V_opposite)[0]
#recompute relative symmetry errors 
LR_diff = np.linalg.norm(V[V_right,:] - gu.reflectX(V[V_opposite[V_right],:]), axis=1)
#shape_scale = V[:,0].max() - V[:,0].min()
#LR_diff = LR_diff_unscaled/shape_scale
sV, sF = get_sphere(V) 
lcol = [1,0,0]
rcol = [0,0,1]
pcol = np.array([lcol,rcol])
maxCol=[1,0,0]
all_vert =  list(range(V.shape[0]))
symm_col =mu.color4plot([0.8, 0.8, 0.8], V)
symm_col[V_right,:] =  np.array([1, 0, 0]) 
symm_col[V_left,:] =  np.array([0, 0, 1]) 
symm_col_pnt = symm_col.copy()
symm_col_pnt[V_middle,:] = 0
shading = dict(wireframe=False, line_color= [0.1,0.1,0.1], width=1000, height=600, wire_width=0.01, flat=False, line_width=0.01)
mu.Out('Left-Right partition')

mu.DrawMeshes(None, [ [sV,  sF, 0.5*symm_col, all_vert, symm_col_pnt], \
                      [V,    F, 0.5*symm_col, all_vert, symm_col_pnt],\
                    ], shifts =(1.5,0,0),mesh_shading = shading,  point_shading=dict(point_size= 0.1))
                    

resolution level=4


### Left-Right partition

Renderer(camera=PerspectiveCamera(aspect=1.6666666666666667, children=(DirectionalLight(color='white', intensi…

In [6]:
V_right_slice = [[],[]]
V_left_slice  = [[],[]]
V_right_slice[0] = V_right[ np.nonzero( sV[V_right,1] >  0 )]
V_right_slice[1] = V_right[ np.nonzero( sV[V_right,1] <= 0 )]
V_left_slice[0]  = V_left[ np.nonzero(  sV[V_left,1]    > 0  ) ]
V_left_slice[1]  = V_left[ np.nonzero(  sV[V_left,1]   <= 0  ) ]

slice_col = symm_col.copy()
slice_col[V_right_slice[0] ] = np.array([1,1,0])
slice_col[V_left_slice[0]  ] = np.array([0,1,1])
slice_col_pnt = slice_col.copy()
slice_col_pnt[V_middle,:] = 0

mu.Out('Left-Right partition with top, bottom horizontal slices')
mu.DrawMeshes(None, [ [sV,  sF, 0.5*slice_col, all_vert, slice_col_pnt], \
                      [V,    F, 0.5*slice_col, all_vert, slice_col_pnt],\
                    ], shifts =(1.5,0,0),mesh_shading = shading,  point_shading=dict(point_size= 0.1))
mu.Out('Left-Right partition with top and bottom slices')


### Left-Right partition with top, bottom horizontal slices

Renderer(camera=PerspectiveCamera(aspect=1.6666666666666667, children=(DirectionalLight(color='white', intensi…

### Left-Right partition with top and bottom slices

In [26]:
#check if the two methods return the  same partitioning 
def is_np_equal(A,B):
    return (A.shape == B.shape) and np.all(A==B)

from copy import deepcopy
data_cp = deepcopy(symmetry_data.copy())
V_right_slice0,  V_left_slice0 = gu.add_horizontal_slices_to_symmetry(data_cp, sV)
data_cp = deepcopy(symmetry_data.copy())
V_right_slice,  V_left_slice = gu.add_horizontal_slices_to_symmetry_pro(data_cp, sV,2)
for i in range(2):
    if not is_np_equal(V_right_slice0[i],V_right_slice[i]):
        print(f'right slices #{i} differs')
    if  not is_np_equal(V_left_slice0[i],V_left_slice[i]):
        print(f'left slices #{i} differs')

#generating more slices with new general function 
slice_num =3 # <-- Set it
V_right_slice,  V_left_slice = gu.add_horizontal_slices_to_symmetry_pro(symmetry_data, sV, slice_num)

#color each segment 
slice_col = symm_col.copy()
vert_colors = cm.Set3(np.linspace(0, 1, slice_num))
for si in range(slice_num):
    slice_col[V_right_slice[si],:] =  vert_colors[si,0:3]
    slice_col[V_left_slice[si], :] =  np.array([1,1,1]) - vert_colors[si,0:3]

In [27]:
mu.Out('All possible symmetrization with top-bottom hirizontal cuts')
#all_symm_types = list(itertools.permutations([1,2,3], 2) ) #exclude  symmetrizations that are of the same type for all slices (e.g. (1,1))
all_symm_types = list(itertools.product([1,2,3], repeat = slice_num) )


symmetry_data['V_right_slice'] = V_right_slice
symmetry_data['V_left_slice']  = V_left_slice

shading    = dict(wireframe=False, line_color= [0.1,0.1,0.1], width=1000, height=1500, wire_width=0.01, flat=False, line_width=0.01)
shading_wf = shading.copy()
shading_wf['wireframe'] = True
shadings = [shading, shading, shading_wf, shading_wf]
symm_solid_col = [0,1,0,0.6]
orig_solid_col = [0.5,0.5,0.5,0.6]

draw_in_single_table = True
#for symm_type in all_symm_types:
mesh_list = []
all_symm_types = [(0,)*slice_num] + all_symm_types #add non symmetric mesh to be the first
all_symm_types = all_symm_types[0:5] #take subrange for debug
print(all_symm_types)
for symm_type in all_symm_types:
    Out(f'symmetrization type={symm_type}')
    V_symm   = gu.symmetrize_sliced_mesh(V,symmetry_data, symm_type)
    print(f'Distance from original mesh = {igl.hausdorff(V,F, V_symm,F)}')
    gu.measure_symmetries(V_symm,F, V_right, V_opposite, {'verbose':True} )
    col_symm = gu.symmetrize_sliced_mesh(slice_col,symmetry_data, symm_type)
    col_symm =np.abs(col_symm)
    symm_alpha_col = np.column_stack( (col_symm,  0.6 * np.ones(col_symm.shape[0])) )
    col_symm_pnt = col_symm.copy() #better highlight for middle points 
    col_symm_pnt[V_middle,:] = np.array([0,0,0])
    mesh_list +=  [     [sV,     sF, col_symm], \
                        [V_symm, F,  col_symm],\
                        mu.StackMeshes( ([V,  F, orig_solid_col], [V_symm,    F, symm_alpha_col]) )\
                 ]
        # if draw_in_single_table:
        #     mu.DrawMeshes(None, [ [sV,  sF,  0.5*col_symm, all_vert,   col_symm_pnt], \
        #                         [V_symm,    F, 0.5*col_symm, all_vert, col_symm_pnt],\
        #                         [V,  F, [0,1,0] ], \
        #                         mu.StackMeshes( ([V,  F, orig_solid_col], [V_symm,    F, symm_solid_col]) ), \
        #                     ], shifts =[(1.5,0,0), (1.5,0,0), (1.5,0,0), (0,0,0) ],mesh_shading = shadings,  point_shading=dict(point_size= 0.1))

nrow = len(all_symm_types)
mesh_table = mu.MeshList2Table(mesh_list,nrow, 3, copy_elements =True) #deesn't work without  'copy_elements' because  mesh_list elements point several time  to the same mesh  
mu.DrawMeshTable(None,mesh_table, col_shift=np.array([1.5, 1.5, 0]),\
                                    row_shift = -1.3, mesh_shading = shading,  point_shading = dict(point_size= 0.08) )

### All possible symmetrization with top-bottom hirizontal cuts

[(0, 0, 0), (1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 2, 1)]


### symmetrization type=(0, 0, 0)

Distance from original mesh = 0.004409028157781923
Mean-max Index and geometric Symmetry errors = (0.0306, 0.2730,0.0296)


### symmetrization type=(1, 1, 1)

...

### symmetrization type=(1, 1, 3)

Distance from original mesh = 0.015552056303759613
Mean-max Index and geometric Symmetry errors = (0.0000, 0.0000,0.0000)


### symmetrization type=(1, 2, 1)

Distance from original mesh = 0.015552056303759613
Mean-max Index and geometric Symmetry errors = (0.0000, 0.0000,0.0000)


Renderer(camera=PerspectiveCamera(aspect=0.6666666666666666, children=(DirectionalLight(color='white', intensi…

In [35]:
#Generate command line to  tests MCMR network with all  possible symmetrization 
def is_same_elememts(x):
    return x and [x[0]]*len(x) == list(x)

all_symm_types =  [(0,)*slice_num] +  list(itertools.product([1,2,3], repeat = slice_num) )
#print(all_symm_types)
const_symm_type = []
#print(f'Different symmetrizations per slice"\n')
i =0 
for symm_type in all_symm_types:
    i =i+1
    #print(symm_type)
    if is_same_elememts(symm_type): 
        const_symm_type.append(str(symm_type[0]))
    else:
        symm_type_str =[str(s) for s in symm_type]
        
        symm_arg = " ".join(symm_type_str)
        tag      = "_".join(symm_type_str)
        print(f'{i:02d} bash scripts/test_mcmr_pascal3d.sh <network> <shape_num>   "<classes>"  $datasize  "--qualitative_results --symmetrize {symm_arg}" symmetric_{tag}')

print(f'same symmetrizations per slice: \n' )
for symm_type in const_symm_type:    
    print(f'bash scripts/test_mcmr_pascal3d.sh <network> <shape_num>   "<classes>"  $datasize  "--qualitative_results --symmetrize {symm_type}" symmetric_{symm_type}')



3 bash scripts/test_mcmr_pascal3d.sh <network> <shape_num>   "<classes>"  $datasize  "--qualitative_results --symmetrize 1 1 2" symmetric_1_1_2
4 bash scripts/test_mcmr_pascal3d.sh <network> <shape_num>   "<classes>"  $datasize  "--qualitative_results --symmetrize 1 1 3" symmetric_1_1_3
5 bash scripts/test_mcmr_pascal3d.sh <network> <shape_num>   "<classes>"  $datasize  "--qualitative_results --symmetrize 1 2 1" symmetric_1_2_1
6 bash scripts/test_mcmr_pascal3d.sh <network> <shape_num>   "<classes>"  $datasize  "--qualitative_results --symmetrize 1 2 2" symmetric_1_2_2
7 bash scripts/test_mcmr_pascal3d.sh <network> <shape_num>   "<classes>"  $datasize  "--qualitative_results --symmetrize 1 2 3" symmetric_1_2_3
8 bash scripts/test_mcmr_pascal3d.sh <network> <shape_num>   "<classes>"  $datasize  "--qualitative_results --symmetrize 1 3 1" symmetric_1_3_1
9 bash scripts/test_mcmr_pascal3d.sh <network> <shape_num>   "<classes>"  $datasize  "--qualitative_results --symmetrize 1 3 2" symmetri

In [10]:
print(all_symm_types)

[(0, 0), (1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]
